In [1]:
#import 
import numpy as np
import pandas as pd
#import hyperopt
from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
import re
deck = {"A": "A", "B": "B", "C": "C", "D": "D", "E": "E", "F": "F", "G": "G", "U": "U"}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna("U")
    #dataset['Deck'] = dataset['Deck'].astype(int)
# we can now drop the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [4]:
data = [train_df, test_df]

for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)

In [5]:
#show how many the null value for each column
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
Deck           0
dtype: int64

In [6]:
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [7]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Deck           0
dtype: int64

In [8]:
x = train_df.drop('Survived',axis=1)
y = train_df.Survived

In [9]:
x.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age              int32
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
Deck            object
dtype: object

In [13]:
cate_features_index = np.where(x.dtypes != float)[0]
cate_features_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10], dtype=int64)

In [14]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.80,random_state=1234)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [16]:
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

Learning rate set to 0.075294
0:	learn: 0.7879213	test: 0.7821229	best: 0.7821229 (0)	total: 75.7ms	remaining: 1m 15s
1:	learn: 0.7780899	test: 0.8212291	best: 0.8212291 (1)	total: 87.7ms	remaining: 43.8s
2:	learn: 0.7780899	test: 0.8212291	best: 0.8212291 (1)	total: 95.1ms	remaining: 31.6s
3:	learn: 0.7780899	test: 0.8212291	best: 0.8212291 (1)	total: 110ms	remaining: 27.4s
4:	learn: 0.8089888	test: 0.7988827	best: 0.8212291 (1)	total: 139ms	remaining: 27.7s
5:	learn: 0.8089888	test: 0.7988827	best: 0.8212291 (1)	total: 159ms	remaining: 26.4s
6:	learn: 0.8117978	test: 0.8156425	best: 0.8212291 (1)	total: 175ms	remaining: 24.9s
7:	learn: 0.8146067	test: 0.8156425	best: 0.8212291 (1)	total: 201ms	remaining: 24.9s
8:	learn: 0.8132022	test: 0.8156425	best: 0.8212291 (1)	total: 203ms	remaining: 22.4s
9:	learn: 0.8174157	test: 0.8156425	best: 0.8212291 (1)	total: 216ms	remaining: 21.3s
10:	learn: 0.8188202	test: 0.8156425	best: 0.8212291 (1)	total: 231ms	remaining: 20.8s
11:	learn: 0.818820

97:	learn: 0.9143258	test: 0.8324022	best: 0.8324022 (73)	total: 1.96s	remaining: 18s
98:	learn: 0.9143258	test: 0.8268156	best: 0.8324022 (73)	total: 1.97s	remaining: 17.9s
99:	learn: 0.9143258	test: 0.8212291	best: 0.8324022 (73)	total: 1.99s	remaining: 17.9s
100:	learn: 0.9143258	test: 0.8212291	best: 0.8324022 (73)	total: 2s	remaining: 17.8s
101:	learn: 0.9143258	test: 0.8212291	best: 0.8324022 (73)	total: 2.03s	remaining: 17.9s
102:	learn: 0.9143258	test: 0.8212291	best: 0.8324022 (73)	total: 2.04s	remaining: 17.8s
103:	learn: 0.9157303	test: 0.8156425	best: 0.8324022 (73)	total: 2.07s	remaining: 17.8s
104:	learn: 0.9185393	test: 0.8156425	best: 0.8324022 (73)	total: 2.09s	remaining: 17.8s
105:	learn: 0.9199438	test: 0.8156425	best: 0.8324022 (73)	total: 2.11s	remaining: 17.8s
106:	learn: 0.9199438	test: 0.8156425	best: 0.8324022 (73)	total: 2.13s	remaining: 17.8s
107:	learn: 0.9213483	test: 0.8156425	best: 0.8324022 (73)	total: 2.15s	remaining: 17.8s
108:	learn: 0.9227528	test: 0

201:	learn: 0.9410112	test: 0.8156425	best: 0.8324022 (73)	total: 4.09s	remaining: 16.2s
202:	learn: 0.9424157	test: 0.8156425	best: 0.8324022 (73)	total: 4.11s	remaining: 16.1s
203:	learn: 0.9438202	test: 0.8156425	best: 0.8324022 (73)	total: 4.14s	remaining: 16.1s
204:	learn: 0.9452247	test: 0.8156425	best: 0.8324022 (73)	total: 4.16s	remaining: 16.1s
205:	learn: 0.9452247	test: 0.8156425	best: 0.8324022 (73)	total: 4.18s	remaining: 16.1s
206:	learn: 0.9452247	test: 0.8156425	best: 0.8324022 (73)	total: 4.2s	remaining: 16.1s
207:	learn: 0.9452247	test: 0.8156425	best: 0.8324022 (73)	total: 4.22s	remaining: 16.1s
208:	learn: 0.9452247	test: 0.8156425	best: 0.8324022 (73)	total: 4.24s	remaining: 16s
209:	learn: 0.9452247	test: 0.8212291	best: 0.8324022 (73)	total: 4.26s	remaining: 16s
210:	learn: 0.9452247	test: 0.8212291	best: 0.8324022 (73)	total: 4.28s	remaining: 16s
211:	learn: 0.9466292	test: 0.8212291	best: 0.8324022 (73)	total: 4.3s	remaining: 16s
212:	learn: 0.9466292	test: 0.8

301:	learn: 0.9691011	test: 0.8156425	best: 0.8324022 (73)	total: 6.25s	remaining: 14.5s
302:	learn: 0.9691011	test: 0.8156425	best: 0.8324022 (73)	total: 6.28s	remaining: 14.4s
303:	learn: 0.9705056	test: 0.8156425	best: 0.8324022 (73)	total: 6.3s	remaining: 14.4s
304:	learn: 0.9705056	test: 0.8156425	best: 0.8324022 (73)	total: 6.32s	remaining: 14.4s
305:	learn: 0.9705056	test: 0.8156425	best: 0.8324022 (73)	total: 6.33s	remaining: 14.4s
306:	learn: 0.9705056	test: 0.8156425	best: 0.8324022 (73)	total: 6.36s	remaining: 14.3s
307:	learn: 0.9691011	test: 0.8156425	best: 0.8324022 (73)	total: 6.38s	remaining: 14.3s
308:	learn: 0.9691011	test: 0.8156425	best: 0.8324022 (73)	total: 6.4s	remaining: 14.3s
309:	learn: 0.9691011	test: 0.8212291	best: 0.8324022 (73)	total: 6.42s	remaining: 14.3s
310:	learn: 0.9691011	test: 0.8212291	best: 0.8324022 (73)	total: 6.43s	remaining: 14.3s
311:	learn: 0.9691011	test: 0.8212291	best: 0.8324022 (73)	total: 6.45s	remaining: 14.2s
312:	learn: 0.9705056	t

403:	learn: 0.9845506	test: 0.8156425	best: 0.8324022 (73)	total: 8.32s	remaining: 12.3s
404:	learn: 0.9845506	test: 0.8156425	best: 0.8324022 (73)	total: 8.34s	remaining: 12.3s
405:	learn: 0.9845506	test: 0.8268156	best: 0.8324022 (73)	total: 8.36s	remaining: 12.2s
406:	learn: 0.9845506	test: 0.8268156	best: 0.8324022 (73)	total: 8.38s	remaining: 12.2s
407:	learn: 0.9845506	test: 0.8268156	best: 0.8324022 (73)	total: 8.4s	remaining: 12.2s
408:	learn: 0.9845506	test: 0.8268156	best: 0.8324022 (73)	total: 8.42s	remaining: 12.2s
409:	learn: 0.9845506	test: 0.8212291	best: 0.8324022 (73)	total: 8.44s	remaining: 12.2s
410:	learn: 0.9845506	test: 0.8156425	best: 0.8324022 (73)	total: 8.46s	remaining: 12.1s
411:	learn: 0.9845506	test: 0.8156425	best: 0.8324022 (73)	total: 8.49s	remaining: 12.1s
412:	learn: 0.9845506	test: 0.8212291	best: 0.8324022 (73)	total: 8.51s	remaining: 12.1s
413:	learn: 0.9845506	test: 0.8268156	best: 0.8324022 (73)	total: 8.53s	remaining: 12.1s
414:	learn: 0.9845506	

...

600:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.3s	remaining: 8.18s
601:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.3s	remaining: 8.16s
602:	learn: 0.9971910	test: 0.8324022	best: 0.8324022 (73)	total: 12.4s	remaining: 8.14s
603:	learn: 0.9971910	test: 0.8324022	best: 0.8324022 (73)	total: 12.4s	remaining: 8.11s
604:	learn: 0.9971910	test: 0.8324022	best: 0.8324022 (73)	total: 12.4s	remaining: 8.09s
605:	learn: 0.9971910	test: 0.8324022	best: 0.8324022 (73)	total: 12.4s	remaining: 8.07s
606:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.4s	remaining: 8.05s
607:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.5s	remaining: 8.03s
608:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.5s	remaining: 8.01s
609:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.5s	remaining: 7.99s
610:	learn: 0.9971910	test: 0.8268156	best: 0.8324022 (73)	total: 12.5s	remaining: 7.97s
611:	learn: 0.9971910

694:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.2s	remaining: 6.23s
695:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.2s	remaining: 6.21s
696:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.2s	remaining: 6.18s
697:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.2s	remaining: 6.16s
698:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.14s
699:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.12s
700:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.1s
701:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.08s
702:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.05s
703:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.3s	remaining: 6.03s
704:	learn: 0.9985955	test: 0.8212291	best: 0.8324022 (73)	total: 14.4s	remaining: 6.01s
705:	learn: 0.9985955	

789:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.1s	remaining: 4.27s
790:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.1s	remaining: 4.25s
791:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.1s	remaining: 4.22s
792:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.1s	remaining: 4.2s
793:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.1s	remaining: 4.18s
794:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.2s	remaining: 4.16s
795:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.2s	remaining: 4.14s
796:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.2s	remaining: 4.12s
797:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.2s	remaining: 4.1s
798:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.2s	remaining: 4.08s
799:	learn: 0.9985955	test: 0.8156425	best: 0.8324022 (73)	total: 16.3s	remaining: 4.06s
800:	learn: 0.9985955	t

889:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.2s	remaining: 2.24s
890:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.2s	remaining: 2.23s
891:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.2s	remaining: 2.21s
892:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.2s	remaining: 2.18s
893:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.16s
894:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.14s
895:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.12s
896:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.1s
897:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.08s
898:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.3s	remaining: 2.06s
899:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 18.4s	remaining: 2.04s
900:	learn: 1.0000000	

991:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.2s	remaining: 163ms
992:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.3s	remaining: 143ms
993:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.3s	remaining: 122ms
994:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.3s	remaining: 102ms
995:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.3s	remaining: 81.6ms
996:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.3s	remaining: 61.2ms
997:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.4s	remaining: 40.8ms
998:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.4s	remaining: 20.4ms
999:	learn: 1.0000000	test: 0.8212291	best: 0.8324022 (73)	total: 20.4s	remaining: 0us

bestTest = 0.8324022346
bestIteration = 73

Shrink model to first 74 iterations.


In [23]:
print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest,model.predict(xtest))))

the test accuracy is :0.832402


In [24]:
pred = model.predict(test_df)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':pred})

In [25]:
submission.to_csv('catboost.csv',index=False)